# Detecção de Falhas em Sinais de Sensores com Número Variável de Sensores

**Universidade Federal do Maranhão**  
Professor Dr. Thales Levi Azevedo Valente  
Engenharia da Computação – Fundamentos de Redes Neurais

## Objetivo

Treinar, avaliar e explicar um modelo em TensorFlow/Keras capaz de identificar falhas em sinais de séries temporais de sensores, aceitando qualquer quantidade de sensores sem necessidade de re-treinamento.

## 1. Bibliotecas Utilizadas

Abaixo estão as bibliotecas necessárias para execução do notebook.

In [1]:
# Bibliotecas principais
import os
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, optimizers
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns

## 2. Estatísticas, Análise dos Dados e Justificativas

- **Estatísticas globais** (usadas para normalização):  
  - mínimo = −37 969.1569  
  - máximo = 37 128.0924  
  - Q1 = −29.3986  
  - Q2 = 55.3385  
  - Q3 = 319.5190  
  - média = 31.9291  
  - samplingRate = 1 Hz

- **Justificativa da janela temporal:**  
  - Usaremos `WINDOW = 240` (4 minutos), pois permite capturar padrões temporais relevantes sem perder granularidade.
  - `STRIDE = 60` (1 minuto) para reduzir sobreposição excessiva e manter diversidade nas janelas.

- **Divisão temporal:**  
  - 70% treino, 15% validação, 15% teste, sem embaralhar, para respeitar a ordem temporal.

- **Abordagem escolhida:**  
  - **B. LSTM compartilhada (janela por sensor)**: cada janela é (WINDOW, 1) para cada sensor individualmente, permitindo lidar com número variável de sensores.

## 3. Carregamento e Preparação dos Dados

- Concatenação dos CSVs
- Normalização usando estatísticas globais
- Conversão dos labels para binário (1 = falha, 0 = normal)
- Criação das janelas temporais por sensor
- Divisão temporal correta

In [2]:
# Caminho dos arquivos CSV
csv_files = sorted(glob.glob(os.path.join('dataset', 'dataset_parte_*.csv')))

# Carregar e concatenar todos os CSVs
df = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)

# Identificar colunas de sensores (excluindo Timestamp e label)
sensor_cols = [col for col in df.columns if col.startswith('sensor_')]

# Normalização (Min-Max global)
MIN, MAX = -37969.1569, 37128.0924
df[sensor_cols] = (df[sensor_cols] - MIN) / (MAX - MIN)

# Converter label para binário: 1 se há falha, 0 caso contrário
df['target'] = df['label'].apply(lambda x: 0 if pd.isna(x) or x == '' else 1)

C:\Users\C18270Q\AppData\Local\Temp\ipykernel_21168\474610911.py:5: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)


In [3]:
# Função para criar janelas por sensor
WINDOW = 240
STRIDE = 60

def create_sensor_windows(data, window, stride, sensor_cols):
    X, y = [], []
    for start in range(0, len(data) - window + 1, stride):
        end = start + window
        window_data = data.iloc[start:end]
        for sensor in sensor_cols:
            X.append(window_data[sensor].values.reshape(-1, 1))
            # Se qualquer label na janela for 1, marcamos como falha
            y.append(window_data['target'].max())
    return np.array(X), np.array(y)

X_B, y_B = create_sensor_windows(df, WINDOW, STRIDE, sensor_cols)
print(f'Formato das janelas: {X_B.shape} (amostras, tempo, 1)')
print(f'Proporção de falhas: {np.mean(y_B):.4f}')

Formato das janelas: (342040, 240, 1) (amostras, tempo, 1)
Proporção de falhas: 0.0001


In [4]:
# Divisão temporal (por janelas, já que não há embaralhamento)
n_total_B = X_B.shape[0]
n_train_B = int(0.7 * n_total_B)
n_val_B = int(0.15 * n_total_B)

X_train_B, y_train_B = X_B[:n_train_B], y_B[:n_train_B]
X_val_B, y_val_B = X_B[n_train_B:n_train_B+n_val_B], y_B[n_train_B:n_train_B+n_val_B]
X_test_B, y_test_B = X_B[n_train_B+n_val_B:], y_B[n_train_B+n_val_B:]

print(f'Treino: {X_train_B.shape}, Validação: {X_val_B.shape}, Teste: {X_test_B.shape}')

Treino: (239427, 240, 1), Validação: (51306, 240, 1), Teste: (51307, 240, 1)


## 4. Pipeline tf.data

- Shuffle apenas no treino
- Batching

In [5]:
BATCH_SIZE = 32
ds_train_B = tf.data.Dataset.from_tensor_slices((X_train_B, y_train_B)).shuffle(1024).batch(BATCH_SIZE).prefetch(1)
ds_val_B = tf.data.Dataset.from_tensor_slices((X_val_B, y_val_B)).batch(BATCH_SIZE).prefetch(1)
ds_test_B = tf.data.Dataset.from_tensor_slices((X_test_B, y_test_B)).batch(BATCH_SIZE).prefetch(1)

## 5. Construção do Modelo (LSTM Compartilhada)

- LSTM 1D compartilhada para todos os sensores
- Camadas de normalização e regularização
- Saída binária

In [6]:
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.regularizers import l2

model_B = models.Sequential([
    layers.InputLayer(input_shape=(WINDOW, 1)),
    layers.LSTM(64, return_sequences=True, kernel_initializer=HeNormal()),
    layers.BatchNormalization(),
    layers.LSTM(64, kernel_initializer=HeNormal(), kernel_regularizer=l2(0.01)),
    layers.BatchNormalization(),
    layers.Dense(64, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=l2(0.01)),
    layers.BatchNormalization(),
    layers.Dense(1, activation='sigmoid')
])

model_B.compile(
    optimizer=optimizers.Adam(),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)
model_B.summary()

c:\Users\C18270Q\ATV EXTRA REDES NEURAIS\extra-task\.venv\Lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 240, 64)        │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 240, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 54,913 (214.50 KB)

 Trainable params: 54,529 (213.00 KB)

 Non-trainable params: 384 (1.50 KB)

## 6. Treinamento do Modelo

- ReduceLROnPlateau
- Logs claros

In [ ]:
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', patience=10, factor=0.5, min_lr=1e-6)

history = model_B.fit(
    ds_train_B,
    validation_data=ds_val_B,
    epochs=200,
    callbacks=[reduce_lr]
)

Epoch 1/200
7483/7483 ━━━━━━━━━━━━━━━━━━━━ 1173s 156ms/step - accuracy: 0.9937 - loss: 0.4783 - precision: 0.1114 - recall: 0.7943 - val_accuracy: 1.0000 - val_loss: 6.0122e-06 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 2/200
7483/7483 ━━━━━━━━━━━━━━━━━━━━ 1173s 156ms/step - accuracy: 0.9937 - loss: 0.4783 - precision: 0.1114 - recall: 0.7943 - val_accuracy: 1.0000 - val_loss: 6.0122e-06 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 2/200
7483/7483 ━━━━━━━━━━━━━━━━━━━━ 1300s 174ms/step - accuracy: 0.9991 - loss: 0.0075 - precision: 0.4977 - recall: 0.0293 - val_accuracy: 1.0000 - val_loss: 1.3253e-06 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 3/200
7483/7483 ━━━━━━━━━━━━━━━━━━━━ 1300s 174ms/step - accuracy: 0.9991 - loss: 0.0075 - precision: 0.4977 - recall: 0.0293 - val_accuracy: 1.0000 - val_loss: 1.3253e-06 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learni

## 7. Avaliação em Validação e Teste

- Matriz de confusão
- Curvas ROC/AUC
- Precision, Recall, F1-Score

In [ ]:
# Avaliação em Teste
y_pred = (model_B.predict(ds_test_B) > 0.5).astype(int).flatten()
y_true = y_test_B

print(classification_report(y_true, y_pred, digits=4))
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(4,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predito')
plt.ylabel('Real')
plt.title('Matriz de Confusão - Teste')
plt.show()

## 8. Resultados, Interpretação, Limitações e Ideias Futuras

- **Resultados:** Apresente as métricas obtidas e discuta a performance.
- **Interpretação:** Analise possíveis causas de erros e acertos.
- **Limitações:** O modelo pode não capturar todas as interações complexas, especialmente se houver poucos exemplos de falha.
- **Ideias Futuras:**  
  - Testar abordagem C (espectrogramas) para explorar padrões espectrais.
  - Experimentar diferentes tamanhos de janela e stride.
  - Ajustar hiperparâmetros e técnicas de balanceamento de classes.